## Tarea 5

Escuela de Ingeniería Electrónica<br>
EL5857 Aprendizaje Automático<br>
I Semestre 2023

In [1]:
import os
import tkinter as tk
import pickle
# Some version magic here
from PIL import Image,ImageDraw

try:
    from PIL import ImageOps, Resampling
    resampling_filter = Resampling.LANCZOS
except ImportError:
    resampling_filter = Image.LANCZOS

import numpy as np

In [2]:
class DigitPainter(tk.Frame):
    def __init__(self,master=None):
        super().__init__(master)
        self.master = master
        self.pack()
        
        self.lastX, self.lastY = 0, 0        
        self.pencil=55        
        self.imgside=448
        
        self.h=int((self.pencil-1)/2)
        self.create_widgets()
        self.Modelo_SVM={}
        
    def create_widgets(self):
        self.master.geometry('448x512')  # 448=28x16
        self.master.title("Drawing window")

        self.img = Image.new(mode='L',size=(self.imgside,self.imgside),color=(0)) # 'L' means 8-bits        
        self.painter=ImageDraw.Draw(self.img)
        self.canvas= tk.Canvas(self.master,width=self.imgside,height=self.imgside,bg="black")
               
        self.canvas.bind( "<Button-1>", self.get_x_and_y )
        self.canvas.bind( "<B1-Motion>", self.draw_box )
        self.canvas.pack() #expand = YES, fill = BOTH)
        
        # Botones y acciones
        self.frame = tk.Frame(self.master)
        self.frame.pack()

        self.resetButton=tk.Button(self.frame,text="Reset",command= self.erase_canvas) 
        self.resetButton.pack(side= tk.LEFT)
        
        self.svmButton=tk.Button(self.frame,text="SVM",command= self.predictSVM)
        self.svmButton.pack(side= tk.RIGHT)

        self.annButton=tk.Button(self.frame,text="ANN",command= self.predictANN)
        self.annButton.pack(side= tk.RIGHT)

        self.knnButton=tk.Button(self.frame,text="KNN",command= self.predictKNN)
        self.knnButton.pack(side= tk.RIGHT)

        self.rdfButton=tk.Button(self.frame,text="RDF",command= self.predictRDF)
        self.rdfButton.pack(side= tk.RIGHT)
        
        
    # Called when the button is pressed the first time
    def get_x_and_y(self,event):
        self.lastX,self.lastY = event.x, event.y

        self.svmButton['state']=tk.NORMAL
        self.annButton['state']=tk.NORMAL
        self.knnButton['state']=tk.NORMAL
        self.rdfButton['state']=tk.NORMAL

        self.resetButton['state']=tk.NORMAL
        
    # Called on a drag event
    def draw_box(self,event):
        
        self.canvas.create_line((self.lastX,self.lastY,event.x,event.y),fill='white',width=self.pencil)
        self.canvas.create_oval((event.x-self.h,event.y-self.h,event.x+self.h,event.y+self.h),
                                fill='white',outline='white')
        
        self.painter.line([(self.lastX,self.lastY),(event.x,event.y)],fill='white',width=self.pencil)
        self.painter.ellipse([(event.x-self.h,event.y-self.h),(event.x+self.h,event.y+self.h)],
                               fill='white',outline='white')        
        
        self.lastX,self.lastY=event.x,event.y

        
    #Encargado de borrar todo (para bóton)
    def erase_canvas(self):
    
        self.canvas.delete('all')  
        
        self.svmButton['state']=tk.DISABLED
        self.annButton['state']=tk.DISABLED
        self.knnButton['state']=tk.DISABLED
        self.rdfButton['state']=tk.DISABLED
        
        self.resetButton['state']=tk.DISABLED
    
        self.painter.rectangle([0,0,self.imgside,self.imgside],fill='black',outline='black')
        print("***** Nuevo Número *****")
        
    def getMNISTImage(self):
        return np.array(self.img.resize((28,28),resampling_filter))
        
    # ---------------------------
    # Your code below this point
        
    def loadPredictors(self):
        tipomodelo = 2
        if tipomodelo == 1:
            print("Loading predictors...")
            self.Modelo_SVM = pickle.load(open('Modelos/Modelo_SVM.sav', 'rb'))
            print("Modelo SVM")
            self.Modelo_ANN = pickle.load(open('Modelos/Modelo_ANN.sav', 'rb'))
            print("Modelo ANN")
            self.Modelo_KNN = pickle.load(open('Modelos/Modelo_KNN.sav', 'rb'))
            print("Modelo kNN")
            self.Modelo_RDF = pickle.load(open('Modelos/Modelo_RDF.sav', 'rb'))
            print("Modelo RDF")
        
        elif tipomodelo == 2:
            print("Loading predictors...")
            self.Modelo_SVM = pickle.load(open('Modelos/Modelo_SVM_op.sav', 'rb'))
            print("Modelo SVM")
            self.Modelo_ANN = pickle.load(open('Modelos/Modelo_ANN_op.sav', 'rb'))
            print("Modelo ANN")
            self.Modelo_KNN = pickle.load(open('Modelos/Modelo_KNN_op.sav', 'rb'))
            print("Modelo kNN")
            self.Modelo_RDF = pickle.load(open('Modelos/Modelo_RDF_op.sav', 'rb'))
            print("Modelo RDF")
        # ADD YOUR CODE HERE TO LOAD ALL FOUR PREDICTORS        

    def predictSVM(self):
        print("Predict SVM called")
        with open('Normalizadores/scaler_SVM.pkl', 'rb') as f:
            scaler = pickle.load(f)
        
        img=self.getMNISTImage()
        img=scaler.transform(img.reshape(1,-1))
        pred = self.Modelo_SVM.predict(img)
        print("Prediction:", pred[0])
        # ADD YOUR CODE HERE TO PREDICT WITH SVM

    def predictANN(self):
        print("Predict ANN called")

        with open('Normalizadores/scaler_ANN.pkl', 'rb') as f:
            scaler = pickle.load(f)
        
        img=self.getMNISTImage()
        img=scaler.transform(img.reshape(1,-1))
        pred = self.Modelo_ANN.predict(img)
        print("Prediction:", pred[0])
          
        # ADD YOUR CODE HERE TO PREDICT WITH ANN

        
    def predictKNN(self):
        print("Predict KNN called")
        with open('Normalizadores/scaler_KNN.pkl', 'rb') as f:
            scaler = pickle.load(f)
        
        img=self.getMNISTImage()
        img=scaler.transform(img.reshape(1,-1))
        pred = self.Modelo_KNN.predict(img)
        print("Prediction:", pred[0])
        
        # ADD YOUR CODE HERE TO PREDICT WITH KNN
        
    def predictRDF(self):
        print("Predict RDF called")
        with open('Normalizadores/scaler_RDF.pkl', 'rb') as f:
            scaler = pickle.load(f)
        
        img=self.getMNISTImage()
        img=scaler.transform(img.reshape(1,-1))
        pred = self.Modelo_RDF.predict(img)
        print("Prediction:", pred[0])
        
        # ADD YOUR CODE HERE TO PREDICT WITH RDF        
            

In [3]:
root = tk.Tk()
painter=DigitPainter(master=root)
painter.loadPredictors()
painter.mainloop()

Loading predictors...
Modelo SVM
Modelo ANN
Modelo kNN
Modelo RDF
Predict RDF called
Prediction: 8
Predict KNN called
Prediction: 8
Predict ANN called
Prediction: 1
Predict SVM called
Prediction: 2
***** Nuevo Número *****
Predict SVM called
Prediction: 2
***** Nuevo Número *****
Predict SVM called
Prediction: 2
***** Nuevo Número *****
Predict SVM called
Prediction: 2
***** Nuevo Número *****
Predict SVM called
Prediction: 2
***** Nuevo Número *****
Predict SVM called
Prediction: 2
***** Nuevo Número *****
Predict SVM called
Prediction: 2
***** Nuevo Número *****
Predict SVM called
Prediction: 2
***** Nuevo Número *****
Predict SVM called
Prediction: 2
